In [4]:
import pandas as pd
import numpy as np
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local').config('spark.driver.memory', '16g').appName('Margin_pre_post').getOrCreate()

In [5]:
total_bazooka = pd.read_csv('../raw_data/Excel_files/Bazooka.csv', index_col=0)

In [6]:
total_bazooka.head()

,item_code,item_price_x,max_date,delivery_weeks,sales_pe,sales_pe_err,beta_del_week,beta_del_week_err,intercept,intercept_err,...,Delta_N_Sales,log_x_prime,N_prime_sales,Delta_Margin,Group_flag,Increment,final_price_amount,voucher_value,campaign_start,campaign_end
0,1000086540,349.99,2021-04-28,3,-1.4374,0.2356,-0.0499,0.0098,9.5983,1.4798,...,-0.012720,5.988933,0.840108,0.778171,up,14.000000000000002%,349.99,0.0,2021-04-17,2021-04-28
1,1000050876,569.99,2021-04-18,2,-1.1908,0.3369,-0.0119,0.0089,9.2930,2.1853,...,-0.010538,6.476647,1.556809,5.774286,up,14.000000000000002%,569.99,0.0,2021-04-17,2021-04-28
2,1000184191,104.99,2021-04-27,4,-4.8137,0.8391,-0.0163,0.0412,26.1239,3.8742,...,-0.042599,4.784893,3.025659,0.183779,up,14.000000000000002%,104.99,0.0,2021-04-17,2021-04-28
3,1000027369,699.99,2021-04-15,2,-1.0499,0.3353,-0.0246,0.0122,7.6897,2.1169,...,-0.009291,6.682094,0.624969,0.938321,up,14.000000000000002%,699.99,0.0,2021-04-17,2021-04-28
4,1000008842,369.99,2021-04-24,1,-3.7269,0.5994,-0.0319,0.0291,25.8375,3.8917,...,-0.032981,6.044504,3.278337,4.521200,up,14.000000000000002%,369.99,0.0,2021-04-17,2021-04-28


In [7]:
bazooka_ready = total_bazooka[['item_code', 'pc2','Group_flag']]

In [8]:
s_bazooka = spark.createDataFrame(bazooka_ready)

In [9]:
#retrieve sales
df_sales = spark.read.parquet(
    '/Users/gabriele.sabato/PycharmProjects/raw_data/price_elasticity_model_data/part-*',
    header=True)

In [10]:
df_sales.dtypes

[('item_code', 'string'),
 ('update_date', 'date'),
 ('delivery_weeks', 'bigint'),
 ('item_price', 'double'),
 ('sales', 'bigint')]

In [11]:
s_bazooka.createOrReplaceTempView('bazooka')
df_sales.createOrReplaceTempView('sales')

In [12]:
df_sales.show()

+------------------+-----------+--------------+-----------+-----+
|         item_code|update_date|delivery_weeks| item_price|sales|
+------------------+-----------+--------------+-----------+-----+
|000000000000001000| 2020-02-09|             1|       29.0|   12|
|000000000000001000| 2021-05-17|             5|       29.0|    0|
|000000001000000008| 2019-07-31|             1|      13.99|    0|
|000000001000000008| 2020-02-19|             1|      13.99|    0|
|000000001000000008| 2020-07-31|             2|       6.99|    0|
|000000001000000008| 2020-09-04|             2|       6.99|    0|
|000000001000000013| 2019-08-25|             1|      39.99|    0|
|000000001000000013| 2020-02-29|             1|27.98999999|    4|
|000000001000000013| 2020-03-06|             1|27.98999999|    1|
|000000001000000014| 2019-06-21|             1|74.98999999|    0|
|000000001000000014| 2019-08-07|             1|74.98999999|    0|
|000000001000000014| 2020-04-13|             1|74.98999999|    1|
|000000001

In [13]:
int_query= spark.sql("""SELECT s.item_code,
       CASE WHEN s.update_date >= '2021-04-17' and s.update_date <= '2021-04-28'
        THEN s.sales * (s.item_price/1.19 - b.pc2) END as Margin_before,
       CASE WHEN s.update_date > '2021-04-28' and s.update_date <= '2021-05-10'
        THEN s.sales * (s.item_price* (1-0.14)/1.19 - b.pc2) END as Margin_HD,--applying 14% discount on the price,
        b.Group_flag as flag,
        s.update_date,
        s.item_price,
        s.sales
  from bazooka as b
  inner join sales as s on CAST(s.item_code as bigint) = b.item_code
    where  s.update_date >= '2021-04-17' and s.update_date <= '2021-05-10'
    ORDER BY item_code,update_date""")

In [14]:
int_query.show(50)

+------------------+------------------+------------------+----+-----------+----------+-----+
|         item_code|     Margin_before|         Margin_HD|flag|update_date|item_price|sales|
+------------------+------------------+------------------+----+-----------+----------+-----+
|000000001000007069|               0.0|              null|down| 2021-04-17|    269.99|    0|
|000000001000007069|               0.0|              null|down| 2021-04-18|    269.99|    0|
|000000001000007069|130.52335294117648|              null|down| 2021-04-19|    269.99|    1|
|000000001000007069|               0.0|              null|down| 2021-04-20|    269.99|    0|
|000000001000007069|               0.0|              null|down| 2021-04-21|    269.99|    0|
|000000001000007069|               0.0|              null|down| 2021-04-22|    269.99|    0|
|000000001000007069|               0.0|              null|down| 2021-04-23|    269.99|    0|
|000000001000007069|261.04670588235297|              null|down| 2021-0

In [61]:
margin_calculation = spark.sql("""
WITH margins as (SELECT s.item_code,
       CASE WHEN s.update_date >= '2021-04-17' and s.update_date <= '2021-04-28'
        THEN s.sales * (s.item_price/1.19 - b.pc2) END as Margin_before,
       CASE WHEN s.update_date > '2021-04-28' and s.update_date <= '2021-05-10'
        THEN s.sales * (s.item_price/1.19 - b.pc2) END as Margin_HD,
        b.Group_flag as flag
  from bazooka as b
  inner join sales as s on CAST(s.item_code as bigint) = b.item_code
    where  s.update_date >= '2021-04-17' and s.update_date <= '2021-05-10')
    SELECT item_code,
           SUM(Margin_before) as Tot_Margin_before,
           SUM(Margin_HD) as Tot_Margin_campaign,
           MAX(flag) as Group_flag
            from margins
            GROUP BY item_code""")


In [62]:
margin_calculation.show()


+------------------+------------------+-------------------+----------+
|         item_code| Tot_Margin_before|Tot_Margin_campaign|Group_flag|
+------------------+------------------+-------------------+----------+
|000000001000008827| 1512.329941176471|  1944.424210084034|      down|
|000000001000007325| 684.0110924369749|  513.0083193277312|      down|
|000000001000007079|2047.7404369747896| 2303.7079915966387|      down|
|000000001000034491| 97.95596638655465|                0.0|      down|
|000000001000103647| 432.3385714285715| 1297.0157142857145|        up|
|000000001000008748| 977.4410924369749|                0.0|      down|
|000000001000050883| 698.0238655462186| 1396.0477310924373|      down|
|000000001000118200| 200.4795798319328|                0.0|      down|
|000000001000123763|134.63386554621852| 134.63386554621852|      down|
|000000001000200152| 248.4977647058824| 331.33035294117656|      down|
|000000001000110022|1734.9747899159665|  3036.205882352942|        up|
|00000

In [63]:
margin_calc = margin_calculation.toPandas()

In [64]:
margin_calc = margin_calc.astype({'Tot_Margin_before': float,'Tot_Margin_campaign': float })

In [65]:
margin_calc = margin_calc[margin_calc['Tot_Margin_before'] > 0.00]
margin_calc = margin_calc[margin_calc['Tot_Margin_campaign'] > 0.00]

In [66]:
margin_calc['Rate'] =(margin_calc['Tot_Margin_campaign'] - margin_calc['Tot_Margin_before']) /margin_calc['Tot_Margin_before']*100

In [67]:
margin_calc.head()

,item_code,Tot_Margin_before,Tot_Margin_campaign,Group_flag,Rate
0,000000001000008827,1512.329941,1944.424210,down,28.571429
1,000000001000007325,684.011092,513.008319,down,-25.000000
2,000000001000007079,2047.740437,2303.707992,down,12.500000
4,000000001000103647,432.338571,1297.015714,up,200.000000
6,000000001000050883,698.023866,1396.047731,down,100.000000


In [68]:
margin_calc_down = margin_calc[margin_calc['Group_flag'] == 'down']

In [73]:
margin_calc_down['Rate'].mean()

44.40483266017532

In [74]:
margin_calc[margin_calc['Group_flag'] == 'up']['Rate'].mean()

88.14083604456275